# Utilização do Snorkel 
--- 
### 01. Import da base

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
books_df = pd.read_csv('../data/livros.csv')
books_df.columns = books_df.columns.map(str.lower)
books_df.head(5)

,titulo,genero
0,Iniciação ao Estudo da Administração,Administracao
1,Iniciação a Administração geral,Administracao
2,Iniciação a Administração de pessoal,Administracao
3,Administração de Materiais,Administracao
4,Gestão Ambiental na Empresa,Administracao


--- 

### 02. Análise da Base 

"Uma maneira de compreender o conteúdo de um texto é realizar uma análise das palavras mais essenciais, ou seja, aquelas que ocorrem com maior frequência e possuem maior significado para o tema abordado. 

Devido à natureza da base de dados, os títulos geralmente consistem em poucas palavras e têm uma quantidade limitada de exemplos, o que possibilita uma análise mais subjetiva para identificar quais palavras podem ser relevantes na classificação do tópico principal."

In [3]:
import re
from unidecode import unidecode 
import nltk
from nltk.corpus import stopwords
import spacy

# Download Portuguese stopwords from the NLTK library
nltk.download('stopwords')

def remove_special_characters_and_stopwords(text):
    """
    Removes special characters (except Latin usual characters),
    tokenizes the text, and removes Portuguese stopwords.
    
    Args:
        text (str): The input text to process.
        
    Returns:
        list of str: A list of cleaned and tokenized words without stopwords.
    """
    # Remove special characters (except Latin usual characters)
    clean_text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove accents and diacritics from the text (e.g., converting é to e)
    clean_text = unidecode(clean_text)
    
    # Tokenize the cleaned text into a list of words
    clean_text = clean_text.strip().split()

    # Get the list of Portuguese stopwords
    stopwords_list = set(stopwords.words('portuguese'))
    
    # Remove stopwords from the tokenized words
    clean_text = [word.lower() for word in clean_text if word.lower() not in stopwords_list]

    return clean_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
books_df['alt_title'] = books_df['titulo'].map(remove_special_characters_and_stopwords)

In [5]:
list(books_df['genero'].unique())

['Administracao',
 'Artes',
 'Biologia',
 'Geografia',
 'Historia',
 'Literatura',
 'Matematica']

In [37]:
# Select the 'genero' and 'alt_title' columns from the DataFrame
# and explode the 'alt_title' column to create multiple rows for each title
# Then we can see the most relevant words for each genero
(
 books_df[['genero', 'alt_title']]
 .explode('alt_title')
 
 # Group the data by 'genero', reset the index, and calculate the value counts
 .groupby(by='genero', as_index=False)
 .value_counts()
 
 # Sort the values in descending order first by 'genero' and then by 'count'
 .sort_values(by=['genero', 'count'], ascending=False)
 
 # Group the sorted data by 'genero' and select the top 15 elements for each group
 .groupby('genero')
 .head(15)

# Change the word inside the double quotes for selection
).query('genero=="Matematica"')


,genero,alt_title,count
1673,Matematica,matematica,89
1674,Matematica,fundamentos,16
1675,Matematica,geometria,12
1676,Matematica,elementar,12
1677,Matematica,calculo,11
1678,Matematica,aplicacoes,11
1679,Matematica,grau,8
1680,Matematica,analitica,8
1681,Matematica,2o,7
1682,Matematica,medio,6


#### 02.01. Conclusões de análises: 

Iremos utilizar o pacote Snorkel para data augmentation, que utiliza conceitos simples 
para fazer uma inferência heurística. Após a remoção das stop words e limpeza das palavras, 
é possível selecionar algumas palavras-chave que podem fortemente indicar um gênero.

Essas análises e seleções não possuem critérios exatos. Portanto, caso você esteja vendo 
o código, é possível que você ache uma ou outra palavra mais relevante do que as selecionadas.



| Administração | Artes   | Biologia   | Geografia  | História    | Literatura | Matemática  |
|---------------|---------|------------|------------|-------------|------------|-------------|
| administracao | museu   | biologia   | geografia  | historia    | literatura | matematica  |
| organizacoes  | pintura | seres      | geografico | guerra      | texto      | fundamentos |
| organizacao   | arte    | genetica   | sociedade  | revolucao   | portuguesa | geometria   |
| gestao        | teatro  | vida       | regiao     | anos        | leitura    | calculo     |
| empresa       | museum  | biologicas | territorio | civilizacao | gramatica  | analitica   |
|               | gallery | evolucao   |            | antiga      |            | financeira  |

--- 
## 03. Snorkel

O pacote Snorkel do Python é um sistema que permite a geração rápida de dados de treinamento com supervisão fraca. Ele foi criado para automatizar o processo de criação e gerenciamento de dados de treinamento, permitindo que os usuários rotulem, construam e gerenciem dados de treinamento programaticamente.